## *IMPLEMENTING A <span style="color:red">SINGLE NEURON NEURAL NETWORK USING ONLY PYTHON</span>*

#### **Problem Statement:** *Predicting whether a Person will buy life insurance based on Age and Affordibility *

Note: Using Sigmoid function as an Activation Function (Binary Classification)

In [456]:
## Importing relevant modules
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras

#### The complete Neural Network is defined in the class below utilizing Gradient Descent for Optimization ####

In [447]:
class NN:
    # Initializing 2 weights w1 and w2 along with the bias to random values
    def __init__(self):   
        self.w1=1
        self.w2=1
        self.bias=0

    # Function for weight and bias details
    def get_weights(self):
        return [self.w1,self.w2,self.bias]
        
    # Log loss function 
    def log_loss(self,yact,ypred):
        ep = 1e-15
        ypred_elog = [max(i,ep) for i in ypred] # adjusting 0 values 
        ypred_elog = [min(i,1-ep) for i in ypred_elog] # adjusting 1 values
        ypred_elog = np.array(ypred_elog)
        
        log_loss = -1*np.mean(yact*np.log(ypred_elog)+(1-yact)*np.log(1-ypred_elog))
        return log_loss

    # Defining a Fit method 
    def fit(self,x_train,x1,x2,y_train,epochs,loss_threshold):
        # Note: x1 and x2 refer to parameters of the train set(which here are 'Age' and 'Affordibility')
        self.gradient_descent(x_train,x1,x2,y_train,epochs,loss_threshold)
        print(f"Model Fitted with w1:{self.w1}, w2:{self.w2},bias:{self.bias}")

    # Defining a Predict method
    def predict(self,x_test,x1,x2):
        w_sum = self.w1*x_test[x1]+self.w2*x_test[x2]+self.bias
        predictions = self.sigmoid(w_sum) # calling sigmoid function on the weighted sum of input values
        return predictions  

    # Function for returning sigmoid function's output of input value
    def sigmoid(self,y_pred):
        return 1/(1+np.exp(-y_pred)) # Since, Sigmoid(z) = 1 / (1 + e^(-z))

    # Optimization function (Gradient Descent))
    def gradient_descent(self,x_train,x1,x2,y,epochs,loss_threshold):
        learning_rate = 0.5 # tuned according to requirement
        n = len(x_train[x1]) # No. of elements
        
        for i in range(epochs):
            wsum = self.w1*x_train[x1]+self.w2*x_train[x2]+self.bias
            ypred = self.sigmoid(wsum) # calculating predictions

            # Calculating the derivative term required for updating weights and bias
            w1d = (1/n)*np.dot(np.transpose(x_train[x1]),(ypred-y))
            w2d = (1/n)*np.dot(np.transpose(x_train[x2]),(ypred-y))
            bd = np.mean(ypred-y)
            # Updating Weights and Bias
            self.w1 = self.w1 - learning_rate* w1d
            self.w2 = self.w2 - learning_rate*w2d
            self.bias = self.bias - learning_rate*bd
            # Calculating Loss
            loss = self.log_loss(y,ypred)
            # Displaying Selective Epochs as per need
            if i%100==0:
                print(f"Epoch->{i}--------\nw1:{self.w1},w2:{self.w2},bias:{self.bias},loss:{loss}\n")
            # Printing details when loss threshold is achieved
            if loss<=loss_threshold:
                print(f"Loss Threshold {loss_threshold} achieved!")
                print(f"Epoch->{i}--------\nw1:{self.w1},w2:{self.w2},bias:{self.bias},loss:{loss}\n")
                break
            

In [448]:
# Reading and splitting Data
df = pd.read_csv("insurance_data.csv")
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2, random_state=25)

In [449]:
# Scaling Splits for input balance
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [450]:
# Creating and Fitting our Model
customModel = NN()
customModel.fit(X_train_scaled,'age','affordibility', y_train, epochs=10000,loss_threshold= 0.4631)

Epoch->0--------
w1:0.974907633470177,w2:0.948348125394529,bias:-0.11341867736368583,loss:0.7113403233723417

Epoch->100--------
w1:2.200713131760032,w2:1.2941584023238903,bias:-1.6607009122062801,loss:0.5390680417774752

Epoch->200--------
w1:3.443016970881803,w2:1.4042218624465033,bias:-2.2571369883752723,loss:0.5005011269691375

Epoch->300--------
w1:4.472179522095915,w2:1.438787986553552,bias:-2.707387811922373,loss:0.4750814640632793

Loss Threshold 0.4631 achieved!
Epoch->366--------
w1:5.051047623653049,w2:1.4569794548473887,bias:-2.9596534546250037,loss:0.46293944095888917

Model Fitted with w1:5.051047623653049, w2:1.4569794548473887,bias:-2.9596534546250037


### Compairing our Model with TensorFlow Model: ###

In [457]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=5000)

E:\python_all\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 629ms/step - accuracy: 0.5000 - loss: 0.7113
Epoch 2/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5000 - loss: 0.7110
Epoch 3/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5000 - loss: 0.7106
Epoch 4/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5000 - loss: 0.7102
Epoch 5/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5000 - loss: 0.7098
Epoch 6/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5000 - loss: 0.7094
Epoch 7/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5000 - loss: 0.7091
Epoch 8/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5000 - loss: 0.7087
Epoch 9/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5000 - loss: 0.7083
Epoch 10/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5000 - loss: 0.7079
Epoch 11/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5000 - loss: 0.7076
Epoch 12/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy

**Prediction Value Comparison:**

In [459]:
# TF Model
model.predict(X_test_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


array([[0.70548487],
       [0.3556957 ],
       [0.16827832],
       [0.47801185],
       [0.72606957],
       [0.8294983 ]], dtype=float32)

In [460]:
# Our custom model
customModel.predict(X_test_scaled,'age','affordibility') # Predictions on test data

2     0.705020
10    0.355836
21    0.161599
11    0.477919
14    0.725586
9     0.828987
dtype: float64

**Compairing Model Parameters:**

In [463]:
# TF Model
coef, intercept = model.get_weights()
coef, intercept

(array([[5.060863 ],
        [1.4086521]], dtype=float32),
 array([-2.913703], dtype=float32))

In [464]:
customModel.get_weights()

[5.051047623653049, 1.4569794548473887, -2.9596534546250037]

### We can see both the Models are very similar! ###